In [56]:
import numpy as np

def generate_videos_from_images(images, labels, n_videos, frames, augment_noise=True):
    """
    Create synthetic videos:
      - Each video is 'frames' images sampled (with replacement) from the same class
    Returns:
      videos: uint8 array (n_videos, frames, H, W, C)
      video_labels: int array (n_videos,)
    """
    num_classes = int(labels.max()) + 1
    IMG_H, IMG_W, IMG_C = images.shape[1], images.shape[2], images.shape[3]
    videos = np.zeros((n_videos, frames, IMG_H, IMG_W, IMG_C), dtype=np.uint8)
    video_labels = np.zeros((n_videos,), dtype=np.int32)

    # precompute indices per class
    class_indices = {c: np.where(labels == c)[0] for c in range(num_classes)}

    per_class = n_videos // num_classes
    idx = 0

    for c in range(num_classes):
        idxs = class_indices[c]
        for _ in range(per_class):
            chosen = np.random.choice(idxs, size=frames, replace=True)
            clip = images[chosen].copy()
            if augment_noise:
                # tiny random noise and brightness jitter
                noise = (np.random.randn(*clip.shape) * 4).astype(np.int16)
                clip = clip.astype(np.int16) + noise
                brightness = np.random.randint(-10, 11)
                clip = clip + brightness
                clip = np.clip(clip, 0, 255).astype(np.uint8)
            videos[idx] = clip
            video_labels[idx] = c
            idx += 1

    # handle any remaining videos (if n_videos not divisible by num_classes)
    while idx < n_videos:
        c = np.random.randint(0, num_classes)
        idxs = class_indices[c]
        chosen = np.random.choice(idxs, size=frames, replace=True)
        clip = images[chosen].copy()
        if augment_noise:
            noise = (np.random.randn(*clip.shape) * 4).astype(np.int16)
            clip = clip.astype(np.int16) + noise
            clip = np.clip(clip, 0, 255).astype(np.uint8)
        videos[idx] = clip
        video_labels[idx] = c
        idx += 1

    return videos, video_labels


In [57]:
# quick synthetic test to ensure function runs
images = np.random.randint(0, 256, size=(200, 32, 32, 3), dtype=np.uint8)
labels = np.random.randint(0, 10, size=(200,), dtype=np.int32)

videos, vlabels = generate_videos_from_images(images, labels, n_videos=20, frames=8)
print("videos.shape:", videos.shape)   # expected (20, 8, 32, 32, 3)
print("vlabels.shape:", vlabels.shape)


videos.shape: (20, 8, 32, 32, 3)
vlabels.shape: (20,)


In [58]:
# Assume you already ran: generate_videos_from_images

videos, vlabels = generate_videos_from_images(images, labels, n_videos=200, frames=8)

print("Videos:", videos.shape)       # (200, 8, 32, 32, 3)
print("Labels:", vlabels.shape)      # (200,)


Videos: (200, 8, 32, 32, 3)
Labels: (200,)


In [59]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(videos, vlabels, test_size=0.2, random_state=42, stratify=vlabels)


In [60]:
import tensorflow as tf
from tensorflow.keras import layers, models

num_classes = len(np.unique(vlabels))

model = models.Sequential([
    layers.Conv3D(32, (3,3,3), activation='relu', input_shape=(X_train.shape[1], X_train.shape[2], X_train.shape[3], X_train.shape[4])),
    layers.MaxPooling3D((1,2,2)),
    layers.Conv3D(64, (3,3,3), activation='relu'),
    layers.MaxPooling3D((2,2,2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv3d_12 (Conv3D)              │ (None, 6, 30, 30, 32)  │         2,624 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling3d_8 (MaxPooling3D)  │ (None, 6, 15, 15, 32)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d_13 (Conv3D)              │ (None, 4, 13, 13, 64)  │        55,360 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling3d_9 (MaxPooling3D)  │ (None, 2, 6, 6, 64)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 4608)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 128)            │       589,952 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 649,226 (2.48 MB)

 Trainable params: 649,226 (2.48 MB)

 Non-trainable params: 0 (0.00 B)

In [63]:
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=5,
    batch_size=16
)


Epoch 1/5
10/10 ━━━━━━━━━━━━━━━━━━━━ 5s 424ms/step - accuracy: 0.0829 - loss: 2.3032 - val_accuracy: 0.1000 - val_loss: 2.3026
Epoch 2/5
10/10 ━━━━━━━━━━━━━━━━━━━━ 6s 496ms/step - accuracy: 0.0829 - loss: 2.3033 - val_accuracy: 0.1000 - val_loss: 2.3026
Epoch 3/5
10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 397ms/step - accuracy: 0.0829 - loss: 2.3033 - val_accuracy: 0.1000 - val_loss: 2.3026
Epoch 4/5
10/10 ━━━━━━━━━━━━━━━━━━━━ 5s 401ms/step - accuracy: 0.0829 - loss: 2.3033 - val_accuracy: 0.1000 - val_loss: 2.3026
Epoch 5/5
10/10 ━━━━━━━━━━━━━━━━━━━━ 6s 492ms/step - accuracy: 0.0829 - loss: 2.3033 - val_accuracy: 0.1000 - val_loss: 2.3026


In [62]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# Predictions
y_pred = np.argmax(model.predict(X_test), axis=1)

# Metrics
print("Classification Report:")
print(classification_report(y_test, y_pred))

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))


2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 113ms/step
Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         4
           1       0.00      0.00      0.00         4
           2       0.00      0.00      0.00         4
           3       0.00      0.00      0.00         4
           4       0.10      1.00      0.18         4
           5       0.00      0.00      0.00         4
           6       0.00      0.00      0.00         4
           7       0.00      0.00      0.00         4
           8       0.00      0.00      0.00         4
           9       0.00      0.00      0.00         4

    accuracy                           0.10        40
   macro avg       0.01      0.10      0.02        40
weighted avg       0.01      0.10      0.02        40

Confusion Matrix:
[[0 0 0 0 4 0 0 0 0 0]
 [0 0 0 0 4 0 0 0 0 0]
 [0 0 0 0 4 0 0 0 0 0]
 [0 0 0 0 4 0 0 0 0 0]
 [0 0 0 0 4 0 0 0 0 0]
 [0 0 0 0 4 0 0 0 0 0]
 [0 0 0 0 4 0 0 0 0 0]


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
